In [1]:
!pip install xgboost joblib

In [2]:
from google.colab import files
uploaded = files.upload()

Saving train.csv.xls to train.csv.xls


In [4]:
import pandas as pd

df = pd.read_csv("train.csv.xls")
df.to_csv("mobile_price.csv", index=False)

df.head()

,battery_power,blue,clock_speed,dual_sim,fc,four_g,int_memory,m_dep,mobile_wt,n_cores,...,px_height,px_width,ram,sc_h,sc_w,talk_time,three_g,touch_screen,wifi,price_range
0,842,0,2.2,0,1,0,7,0.6,188,2,...,20,756,2549,9,7,19,0,0,1,1
1,1021,1,0.5,1,0,1,53,0.7,136,3,...,905,1988,2631,17,3,7,1,1,0,2
2,563,1,0.5,1,2,1,41,0.9,145,5,...,1263,1716,2603,11,2,9,1,1,0,2
3,615,1,2.5,0,0,0,10,0.8,131,6,...,1216,1786,2769,16,8,11,1,0,0,2
4,1821,1,1.2,0,13,1,44,0.6,141,2,...,1208,1212,1411,8,2,15,1,1,0,1


In [5]:
import numpy as np
import joblib

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score, roc_auc_score, precision_score, recall_score, f1_score, matthews_corrcoef
from sklearn.preprocessing import label_binarize

from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier

# Features and target
X = df.drop("price_range", axis=1)
y = df["price_range"]

# Split
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)

# Scaling
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

# Save scaler
joblib.dump(scaler, "scaler.pkl")

# Models dictionary
models = {
    "Logistic Regression": LogisticRegression(max_iter=2000),
    "Decision Tree": DecisionTreeClassifier(),
    "KNN": KNeighborsClassifier(),
    "Naive Bayes": GaussianNB(),
    "Random Forest": RandomForestClassifier(),
    "XGBoost": XGBClassifier(eval_metric='mlogloss')
}

results = []

# For multiclass AUC
y_test_bin = label_binarize(y_test, classes=[0,1,2,3])

for name, model in models.items():
    print(f"Training {name}...")

    model.fit(X_train, y_train)

    y_pred = model.predict(X_test)
    y_prob = model.predict_proba(X_test)

    accuracy = accuracy_score(y_test, y_pred)
    auc = roc_auc_score(y_test_bin, y_prob, multi_class='ovr')
    precision = precision_score(y_test, y_pred, average='weighted')
    recall = recall_score(y_test, y_pred, average='weighted')
    f1 = f1_score(y_test, y_pred, average='weighted')
    mcc = matthews_corrcoef(y_test, y_pred)

    results.append([name, accuracy, auc, precision, recall, f1, mcc])

    joblib.dump(model, f"{name}.pkl")

# Show results
results_df = pd.DataFrame(results, columns=[
    "Model", "Accuracy", "AUC", "Precision", "Recall", "F1", "MCC"
])

results_df

Training Logistic Regression...
Training Decision Tree...
Training KNN...
Training Naive Bayes...
Training Random Forest...
Training XGBoost...


,Model,Accuracy,AUC,Precision,Recall,F1,MCC
0,Logistic Regression,0.9750,0.999591,0.975946,0.9750,0.975020,0.966875
1,Decision Tree,0.8375,0.889755,0.843090,0.8375,0.837224,0.784773
2,KNN,0.5300,0.762890,0.569762,0.5300,0.540707,0.378880
3,Naive Bayes,0.7975,0.955970,0.806132,0.7975,0.799422,0.731329
4,Random Forest,0.9025,0.984430,0.904446,0.9025,0.902951,0.870147
5,XGBoost,0.9050,0.991322,0.906155,0.9050,0.905007,0.873522


In [6]:
files.download("scaler.pkl")

for model_name in models.keys():
    files.download(f"{model_name}.pkl")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

...

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [7]:
results_df.to_csv("model_results.csv", index=False)
files.download("model_results.csv")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>